# LangChain uma Padronização do Trabalho

Neste caderno vamos explorar os conceitos iniciais de LLM sob a perspectiva do Langchain. Considerando a experiência anterior em interações diretas em prompts e tratamentos de saída manualmente a partir de API, vejamos como esta biblioteca pode ser útil.

# Conceitos Básicos

---


# Parte 1 - Prompt

## Preparando o Ambiente

Para iniciar um trabalho sobre a API da OpenAI, vamos primeiro importar a biblioteca python da openai. Para facilitar o carregamento seguro da chave de API utilizada, também utilizaremos a biblioteca python-dotenv.

In [ ]:
%pip install openai
%pip install --upgrade langchain
%pip install langchain-community

In [ ]:
# Setup environment and import required libraries
import openai

In [ ]:
%set_env OPENAI_API_KEY=#Your API Key here

In [ ]:
llm_model="gpt-3.5-turbo"

Neste momento já temos o necessário para comunicar com a API da OpenAI.
Para avançar, vamos criar uma função auxiliar que irá completar as frases que enivemos nas chamadas:

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

## Modelo de Prompt


In [ ]:
template_string = """Traduza o texto \
delimitado entre três aspas simples ``` \
em um estilo que é {style}. \
texto: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """Português brasileiro \
em um tom calmo e respeitoso
"""

In [ ]:
customer_email = """
[URGNT//0v3rL0@d]\
S3rv3rs, t3@mw0rk n33d3d!\
D@ !mpress0r@ d0 3$kr!t0 t@ d0wn.\
T0d@ m1nh@ r00t3c0 d3 @cc3$$ 3$t@ b10ckd.\
Pr3c1s0 d3 @ss1st3nc!@ m@x!m@ p/@ s01v3r ess@ br3@k.\
S3 vc c0nhec3 @lgum@ m@nd1ng@ p/@ bur1t@r d3v1c3s 0u h@cks d3 r3d3 pr@ !mpress0r@s, ch3g@ j@!\
C0nt@t0 r@p!d0 é v!t@l. S3m m@!s.hen.
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
service_reply = """Olá caro cliente, \
acredito que está tudo estável na rede \
por favor verifique se você está na wifi da empresa \
também tente reiniciar a máquina \
um pouco de fé sempre ajuda \
Boa sorte! Sigo à disposição!
"""

In [ ]:
service_style_pirate = """\
estilo criptográfico hacker de escrever, com girias e codigo \
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

## Prompt
Vamos adaptar o caso de uso realizado para classificar o tópico em um atendimento.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Note que as linhas sobre formatação foram intencionalmente
# removidas quando comparadas à mensagem original do lab_01

system_message = f"""
Você vai receber perguntas de atendimento a clientes. \
Classifique cada pergunta em uma categoria primária \
e uma categoria secundária.

Categorias Primárias:
1. Hardware
2. Software
3. Rede e Conectividade
4. Segurança
5. Suporte e Manutenção

Categorias secundárias de 1. Hardware:
1.1: Computadores
1.2: Periféricos
1.3: Componentes Internos

Categorias secundárias de 2. Software:
2.1: Sistemas Operacionais
2.2: Aplicativos
2.3: Sistemas Internos

Categorias secundárias de 3. Rede e Conectividade:
3.1: Internet
3.2: Intranet
3.3: Telefonia e VoIP

Categorias secundárias de 4. Segurança:
4.1: Acesso
4.2: Segurança de Dados
4.3: Segurança Física

Categoria secundárias 5. Suporte e Manutenção
5.1: Atualizações
5.2: Manutenção Preventiva
5.3: Reparos
"""

In [ ]:
user_input_template = "Pergunta de atendimento: {input}"

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_input_template)
])

In [ ]:
input="meu teclado nao está funcionando. eu consigo acessar usando um teclado virtual mas as teclas físicas nao funcionam"

In [ ]:
messages = prompt.format_messages(input=input)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
class ClassExample(BaseModel):
    primaria: str = Field(description="Categoria Primária")
    secundaria: str = Field(description="Categoria Secundária, identificada como subcategoria da primária.")

# Parte 2 - Parser

In [ ]:
output_parser = JsonOutputParser(pydantic_object=ClassExample)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
user_input_template_formatted = """\
Pergunta de atendimento: {input} \
{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_input_template_formatted)
])

messages = prompt.format_messages(input=user_input, format_instructions=format_instructions)

In [ ]:
print(messages[1].content)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
print(output_dict)

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('primaria')

## Chains (uma espiadinha)

Um dos pontos mais importantes do Langchain. Permite conectar outros elementos da ferramenta em cadeia para processar o conteúdo.
O poder delas é que podem ser executadas sobre várias entradas (inputs).

Em nosso exemplo anterior toda a execução pode ser resumida em uma única cadeia de prompt, modelo e transcrição de saída.

In [ ]:
chain = prompt | chat | output_parser

In [ ]:
chain.invoke({
    "input": "meu teclado nao está funcionando. eu consigo acessar usando um teclado virtual mas as teclas físicas nao funcionam",
    "format_instructions": format_instructions
})

Vamos abordar o conceito e diferentes modos de cadeias no lab_04.